## Packages

In [1]:
import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import re
import codecs
from sklearn.manifold import TSNE
import numpy as np
import multiprocessing
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import chardet

from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('stopwords')

from datetime import datetime
from datetime import timedelta

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\h1648\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\h1648\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\h1648\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\h1648\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Clean function and Embedding

In [2]:
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
word_vectors = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [3]:
stop_words = set(stopwords.words('english'))
def clean_text(str_list, lemmatize = True):
    clean_list = []
    wordnum = []
    charnum = []
    for text in str_list:
        text = re.sub(r'#', '', text)
        text = re.sub(r'@\S+', '', text)
        text = re.sub(r'http\S+', '', text)
        text.encode('ascii', 'xmlcharrefreplace')
        words = word_tokenize(text)
        
        clean_words = []
        
        for word in words:
            if (len(word) > 1) and (re.match(r'^\w+$', word)):
                
                if lemmatize == True:
                    lemmatizer = WordNetLemmatizer()
                    word1 = lemmatizer.lemmatize(word)
                    
                    clean_words.append(word1.lower())
        
        clean_text = ' '.join(clean_words)
        clean_text2 = ''.join(clean_words)
        clean_list.append(clean_text)

    return clean_list

In [53]:
tweet = pd.read_csv('lakersnation.csv',index_col = 0)
result = pd.read_csv('gameresult.csv')
tweetr = pd.read_csv('reporter.csv',index_col = 0)
log = pd.read_csv('log.csv')

In [54]:
game1 = log.iloc[1:3,5:]
game1.index = range(2)

In [55]:
gamess = log.iloc[1:82,5:]
gamess.index = range(81)

In [39]:
gamess.shape, game1.shape

((81, 16), (2, 16))

In [56]:
game0 = pd.concat([game1,gamess],axis = 0)
game0.index = range(83)

In [57]:
game = game0.drop(1)
game.index = range(82)

In [58]:
game

,Team,Team.1,Team.2,Team.3,Team.4,Team.5,Team.6,Team.7,Opponent,Opponent.1,Opponent.2,Opponent.3,Opponent.4,Opponent.5,Opponent.6,Opponent.7
0,0.426,0.25,25,48,23,12,4,21,0.455,0.356,23,48,31,11,4,18
1,0.426,0.25,25,48,23,12,4,21,0.455,0.356,23,48,31,11,4,18
2,0.351,0.2,26,38,23,13,5,10,0.463,0.31,31,58,18,7,7,22
3,0.446,0.182,20,40,26,9,9,13,0.44,0.37,27,52,19,6,5,19
4,0.444,0.267,16,47,25,8,4,14,0.457,0.265,21,54,34,12,5,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,0.5,0.409,30,53,28,10,5,16,0.432,0.333,19,39,25,10,4,16
78,0.559,0.3,28,51,36,4,8,5,0.431,0.382,10,44,23,4,4,8
79,0.5,0.353,35,44,27,6,7,11,0.522,0.455,31,41,29,4,3,12
80,0.506,0.414,24,44,27,4,3,14,0.524,0.444,27,38,23,9,4,12


## Game results

In [59]:
result

,G,Date,Start (ET),Unnamed: 3,Unnamed: 4,Unnamed: 5,Opponent,Unnamed: 7,Unnamed: 8,Tm,Opp,W,L,Streak,Notes
0,1,Tue Oct 18 2022,10:00p,NaN,Box Score,@,Golden State Warriors,L,NaN,109,123,0,1,L 1,NaN
1,2,Thu Oct 20 2022,10:00p,NaN,Box Score,NaN,Los Angeles Clippers,L,NaN,97,103,0,2,L 2,NaN
2,3,Sun Oct 23 2022,3:30p,NaN,Box Score,NaN,Portland Trail Blazers,L,NaN,104,106,0,3,L 3,NaN
3,4,Wed Oct 26 2022,10:00p,NaN,Box Score,@,Denver Nuggets,L,NaN,99,110,0,4,L 4,NaN
4,5,Fri Oct 28 2022,8:00p,NaN,Box Score,@,Minnesota Timberwolves,L,NaN,102,111,0,5,L 5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,79,Tue Apr 4 2023,9:00p,NaN,Box Score,@,Utah Jazz,W,OT,135,133,41,38,W 4,NaN
79,80,Wed Apr 5 2023,10:00p,NaN,Box Score,@,Los Angeles Clippers,L,NaN,118,125,41,39,L 1,NaN
80,81,Fri Apr 7 2023,10:30p,NaN,Box Score,NaN,Phoenix Suns,W,NaN,121,107,42,39,W 1,NaN
81,82,Sun Apr 9 2023,3:30p,NaN,Box Score,NaN,Utah Jazz,W,NaN,128,117,43,39,W 2,NaN


In [60]:
result = result[:82]
result2 = result[['Date','Start (ET)','Unnamed: 7']]
result2.tail()

,Date,Start (ET),Unnamed: 7
77,Sun Apr 2 2023,7:00p,W
78,Tue Apr 4 2023,9:00p,W
79,Wed Apr 5 2023,10:00p,L
80,Fri Apr 7 2023,10:30p,W
81,Sun Apr 9 2023,3:30p,W


In [61]:
time = []
for i in range(len(result2['Start (ET)'])):
  time.append(np.array(result2['Start (ET)'])[i].split('p')[0])

In [62]:
gamestart = []
gameend = []
for i in range(len(result2['Date'])):
  gamestart.append(datetime.strptime(np.array(result2['Date'])[i]+time[i], '%a %b %d %Y%H:%M')+ timedelta(hours = 9))
  gameend.append(datetime.strptime(np.array(result2['Date'])[i]+time[i], '%a %b %d %Y%H:%M')+ timedelta(hours = 11, minutes = 30))

In [63]:
result2['Start time PT'] = gamestart
result2['End time PT'] = gameend

C:\Users\h1648\AppData\Local\Temp\ipykernel_15520\198036090.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result2['Start time PT'] = gamestart
C:\Users\h1648\AppData\Local\Temp\ipykernel_15520\198036090.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result2['End time PT'] = gameend


In [64]:
result3 = result2[['Unnamed: 7','Start time PT','End time PT']]

In [65]:
ats = [7.5,5,-4,5,8,2.5,2.5,-3.5,5.5,7,3,4.5,5,-6,-6,9.5,-3.5,1.5,
       -3.5,-6,8.5,2,5.5,12.5,5,-5.5,4,1.5,-3,11.5,8.5,-2.5,7.5,2.5,
       8.5,6,1.5,8.5,3,7.5,12.5,2.5,5.5,-7.5,-2.5,7,6,4.5,-7.5,8,9,2,
       -2.5,1.5,-6.5,8,5.5,2,-4.5,-6,4.5,9,3.5,1.5,5,1.5,3,-2.5,2,-2.5,
       -4,-6.5,-1.5,-3.5,-5,-3.5,-2,-12,-11,-1,-13,-16.5
       ]
result3['ATS'] = ats

C:\Users\h1648\AppData\Local\Temp\ipykernel_15520\4124997735.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result3['ATS'] = ats


In [66]:
result3.index = range(82)
game.index = range(82)

In [67]:
result4 = pd.concat([result3, game], axis=1)

In [68]:
result4

,Unnamed: 7,Start time PT,End time PT,ATS,Team,Team.1,Team.2,Team.3,Team.4,Team.5,Team.6,Team.7,Opponent,Opponent.1,Opponent.2,Opponent.3,Opponent.4,Opponent.5,Opponent.6,Opponent.7
0,L,2022-10-18 19:00:00,2022-10-18 21:30:00,7.5,0.426,0.25,25,48,23,12,4,21,0.455,0.356,23,48,31,11,4,18
1,L,2022-10-20 19:00:00,2022-10-20 21:30:00,5.0,0.426,0.25,25,48,23,12,4,21,0.455,0.356,23,48,31,11,4,18
2,L,2022-10-23 12:30:00,2022-10-23 15:00:00,-4.0,0.351,0.2,26,38,23,13,5,10,0.463,0.31,31,58,18,7,7,22
3,L,2022-10-26 19:00:00,2022-10-26 21:30:00,5.0,0.446,0.182,20,40,26,9,9,13,0.44,0.37,27,52,19,6,5,19
4,L,2022-10-28 17:00:00,2022-10-28 19:30:00,8.0,0.444,0.267,16,47,25,8,4,14,0.457,0.265,21,54,34,12,5,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,W,2023-04-02 16:00:00,2023-04-02 18:30:00,-12.0,0.5,0.409,30,53,28,10,5,16,0.432,0.333,19,39,25,10,4,16
78,W,2023-04-04 18:00:00,2023-04-04 20:30:00,-11.0,0.559,0.3,28,51,36,4,8,5,0.431,0.382,10,44,23,4,4,8
79,L,2023-04-05 19:00:00,2023-04-05 21:30:00,-1.0,0.5,0.353,35,44,27,6,7,11,0.522,0.455,31,41,29,4,3,12
80,W,2023-04-07 19:30:00,2023-04-07 22:00:00,-13.0,0.506,0.414,24,44,27,4,3,14,0.524,0.444,27,38,23,9,4,12


## Twiter(LakersNation)

In [69]:
tweet

,Date,User,Tweet
0,2023-04-12 23:02:00+00:00,LakersNation,Dennis Schroder was happy to redeem himself la...
1,2023-04-12 22:05:00+00:00,LakersNation,LeBron James had high praise for Rui Hachimura...
2,2023-04-12 21:02:00+00:00,LakersNation,Anthony Davis explained what happened on the f...
3,2023-04-12 20:01:00+00:00,LakersNation,The rest of the schedule and TV info. for the ...
4,2023-04-12 19:02:00+00:00,LakersNation,LeBron James enjoyed being back in a 'playoff ...
...,...,...,...
8128,2022-10-18 16:02:00+00:00,LakersNation,Juan Toscano-Anderson discussed the difference...
8129,2022-10-18 15:03:00+00:00,LakersNation,#Lakers will be without Dennis Schroder for ar...
8130,2022-10-18 14:04:00+00:00,LakersNation,Jeanie Buss on why the Lakers decided to exten...
8131,2022-10-18 13:05:00+00:00,LakersNation,Good news: Anthony Davis will not begin the se...


In [70]:
tweettime = []

for i in range(len(tweet['Date'])):
  tweettime.append(datetime.strptime(np.array(tweet['Date'])[i].split('+')[0], '%Y-%m-%d %H:%M:%S')- timedelta(hours = 7))

tweet['PT Date'] = tweettime
tweet2 = tweet.drop('Date',axis = 1)

ingameindex = []
for i in range(len(tweet2['PT Date'])):
  for j in range(len(gameend)):
    if result4['Start time PT'][j]<tweet2['PT Date'][i] and tweet2['PT Date'][i]<result4['End time PT'][j]:
      ingameindex.append(i)

In [71]:
tweet3 = tweet2.drop(ingameindex)
tweet3['clean_tweet'] = clean_text(tweet3['Tweet'], lemmatize = True)
tweet3.index = range(len(tweet3['PT Date']))
tweet3

,User,Tweet,PT Date,clean_tweet
0,LakersNation,Dennis Schroder was happy to redeem himself la...,2023-04-12 16:02:00,dennis schroder wa happy to redeem himself las...
1,LakersNation,LeBron James had high praise for Rui Hachimura...,2023-04-12 15:05:00,lebron james had high praise for rui hachimura...
2,LakersNation,Anthony Davis explained what happened on the f...,2023-04-12 14:02:00,anthony davis explained what happened on the f...
3,LakersNation,The rest of the schedule and TV info. for the ...,2023-04-12 13:01:00,the rest of the schedule and tv info for the l...
4,LakersNation,LeBron James enjoyed being back in a 'playoff ...,2023-04-12 12:02:00,lebron james enjoyed being back in atmosphere ...
...,...,...,...,...
4592,LakersNation,Juan Toscano-Anderson discussed the difference...,2022-10-18 09:02:00,juan discussed the difference he noticed playi...
4593,LakersNation,#Lakers will be without Dennis Schroder for ar...,2022-10-18 08:03:00,lakers will be without dennis schroder for aro...
4594,LakersNation,Jeanie Buss on why the Lakers decided to exten...,2022-10-18 07:04:00,jeanie buss on why the lakers decided to exten...
4595,LakersNation,Good news: Anthony Davis will not begin the se...,2022-10-18 06:05:00,good news anthony davis will not begin the sea...


### Every Pre-Game News

In [72]:
win = []
all = []
bet = []
gamedata = []
for i in range(len(result4['Start time PT'])):

  
  if i == 0:
  
    for j in range(len(tweet3['clean_tweet'])):
      
      if tweet3['PT Date'][j]<result4['Start time PT'][i]:
        if tweet3['clean_tweet'][j] != '':
          
          all.append(tweet3['clean_tweet'][j])
          win.append(result4['Unnamed: 7'][i])
          bet.append(ats[i])
          gamedata.append(result4.iloc[i,4:])

  if i > 0:

    for j in range(len(tweet3['clean_tweet'])):
      
      if tweet3['PT Date'][j]<result4['Start time PT'][i] and tweet3['PT Date'][j]>result4['End time PT'][i-1]:
        if tweet3['clean_tweet'][j] != '':
          
          all.append(tweet3['clean_tweet'][j])
          win.append(result4['Unnamed: 7'][i])
          bet.append(ats[i])
          gamedata.append(result4.iloc[i,4:])

In [73]:
#82場開賽前推文
len(all)

4405

In [74]:
data = pd.DataFrame(index=range(len(all)), columns=range(300))
nullindex = []
for index, text in enumerate(all):
    vectors = []
    for i in range(len(word_tokenize(text))):
        try:
            vector = word_vectors[word_tokenize(text)[i]]
            vectors.append(vector)
        except KeyError:
            vectors.append(np.repeat(0,300))
    if vectors == []:
      print('Null',index)
      nullindex.append(index)
      vectors = np.repeat(0.0,300)
    else:
      vectors = np.nanmean(np.array(vectors),axis = 0).astype('float') 
    data.iloc[index,:] = vectors
    

data

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.019775,0.061937,0.022631,0.084063,-0.023963,-0.029026,0.004819,-0.127755,-0.001178,0.066322,...,-0.011506,-0.004623,-0.017552,-0.025624,0.022446,0.008359,-0.022235,-0.091242,0.016158,-0.062945
1,0.038586,0.086561,-0.01846,0.064128,0.027646,-0.06014,-0.036858,-0.13718,0.068536,0.136576,...,-0.026162,0.042833,-0.076629,0.033366,0.026442,-0.012427,-0.025004,-0.060628,0.026523,-0.028863
2,0.026253,-0.049389,0.019547,0.069366,-0.050223,-0.0121,-0.030792,-0.039505,0.070557,0.074436,...,0.096649,-0.02478,-0.085144,-0.014648,0.005463,-0.063538,-0.042902,-0.045166,0.02047,0.031281
3,0.061963,0.0257,0.01154,0.033382,0.005153,0.00931,0.030509,-0.020597,0.050814,0.045369,...,-0.013832,-0.007953,-0.101259,0.044649,-0.061057,-0.018164,0.030347,-0.03006,-0.012313,-0.026378
4,0.001442,0.035405,0.04308,0.083834,0.004107,-0.018592,0.038923,-0.0757,0.06779,0.064312,...,-0.086073,0.061743,-0.103396,0.020124,-0.07637,-0.029695,0.024275,-0.013826,-0.023557,-0.050586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4400,-0.022892,0.089437,0.010002,0.124243,-0.00769,-0.040771,-0.020631,-0.126904,0.085164,0.053036,...,-0.030127,0.060531,-0.031807,-0.002775,-0.035917,0.003446,-0.070044,-0.08693,0.01755,-0.011502
4401,0.030564,0.070486,0.035077,0.050976,-0.017258,-0.030233,0.0017,-0.10391,0.053533,0.040152,...,-0.060163,0.044028,-0.073609,-0.006634,-0.046818,0.023438,-0.015083,-0.056964,0.047622,-0.03211
4402,0.054208,0.061035,0.020148,0.086161,-0.022404,-0.036717,0.012667,-0.070506,0.041775,0.048925,...,-0.048153,0.055533,-0.094083,-0.012409,-0.086988,0.014926,-0.014898,-0.053585,0.057474,-0.000676
4403,0.072021,0.065837,-0.023275,0.002767,0.190104,0.020833,-0.043396,-0.079915,0.050415,0.12443,...,0.047445,-0.050863,0.026855,-0.025391,-0.176107,-0.066569,0.060832,-0.013346,-0.166992,0.03656


In [75]:
gameframe = pd.DataFrame(gamedata)

In [76]:
data.index = range(len(all))
gameframe.index = range(len(all))

In [77]:
#VETORIZED之後的文本、GAME LOG、ATS合併
data2 = pd.concat([data,gameframe],axis = 1)
data2['ATS'] = bet
data2.shape

(4405, 317)

In [78]:
data2

,0,1,2,3,4,5,6,7,8,9,...,Team.7,Opponent,Opponent.1,Opponent.2,Opponent.3,Opponent.4,Opponent.5,Opponent.6,Opponent.7,ATS
0,0.019775,0.061937,0.022631,0.084063,-0.023963,-0.029026,0.004819,-0.127755,-0.001178,0.066322,...,21,0.455,0.356,23,48,31,11,4,18,7.5
1,0.038586,0.086561,-0.01846,0.064128,0.027646,-0.06014,-0.036858,-0.13718,0.068536,0.136576,...,21,0.455,0.356,23,48,31,11,4,18,7.5
2,0.026253,-0.049389,0.019547,0.069366,-0.050223,-0.0121,-0.030792,-0.039505,0.070557,0.074436,...,21,0.455,0.356,23,48,31,11,4,18,7.5
3,0.061963,0.0257,0.01154,0.033382,0.005153,0.00931,0.030509,-0.020597,0.050814,0.045369,...,21,0.455,0.356,23,48,31,11,4,18,7.5
4,0.001442,0.035405,0.04308,0.083834,0.004107,-0.018592,0.038923,-0.0757,0.06779,0.064312,...,21,0.455,0.356,23,48,31,11,4,18,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4400,-0.022892,0.089437,0.010002,0.124243,-0.00769,-0.040771,-0.020631,-0.126904,0.085164,0.053036,...,15,0.413,0.25,14,47,23,7,10,9,-16.5
4401,0.030564,0.070486,0.035077,0.050976,-0.017258,-0.030233,0.0017,-0.10391,0.053533,0.040152,...,15,0.413,0.25,14,47,23,7,10,9,-16.5
4402,0.054208,0.061035,0.020148,0.086161,-0.022404,-0.036717,0.012667,-0.070506,0.041775,0.048925,...,15,0.413,0.25,14,47,23,7,10,9,-16.5
4403,0.072021,0.065837,-0.023275,0.002767,0.190104,0.020833,-0.043396,-0.079915,0.050415,0.12443,...,15,0.413,0.25,14,47,23,7,10,9,-16.5


In [79]:
#WIN = 1 LOSS = 0
label0 = []
for i in range(len(win)):
  if win[i] == 'W':
    label0.append(int(1))
  if win[i] == 'L':
    label0.append(int(0))
label0 = np.array(label0)
label0.shape

(4405,)

In [80]:
data2 = np.array(data2)
data2.shape

(4405, 317)

## Twitter(LakersReporter)

In [81]:
tweetrtime = []

for i in range(len(tweetr['Date'])):
  tweetrtime.append(datetime.strptime(np.array(tweetr['Date'])[i].split('+')[0], '%Y-%m-%d %H:%M:%S')- timedelta(hours = 7))

tweetr['PT Date'] = tweetrtime
tweetr2 = tweetr.drop('Date',axis = 1)

ingameindexr = []
for i in range(len(tweetr2['PT Date'])):
  for j in range(len(gameend)):
    if result4['Start time PT'][j]<tweetr2['PT Date'][i] and tweetr2['PT Date'][i]<result4['End time PT'][j]:
      ingameindexr.append(i)

In [82]:
tweetr3 = tweetr2.drop(ingameindexr)
tweetr3['clean_tweet'] = clean_text(tweetr3['Tweet'], lemmatize = True)
tweetr3.index = range(len(tweetr3['PT Date']))
tweetr3

,User,Tweet,PT Date,clean_tweet
0,LakersReporter,👇🏻👇🏻,2023-04-12 13:28:37,
1,LakersReporter,"Anthony Davis (24 points, 15 boards, 4 assists...",2023-04-11 23:20:45,anthony davis 24 point 15 board assist block s...
2,LakersReporter,The Lakers have been playing important games t...,2023-04-11 22:52:52,the lakers have been playing important game to...
3,LakersReporter,"Game 1 at Memphis is at 12 p.m. Pacific, on AB...",2023-04-11 22:51:13,game at memphis is at 12 pacific on abc pregam...
4,LakersReporter,"The Lakers, who started 2-10 and were 26-32 mi...",2023-04-11 22:20:53,the lakers who started and were midway through...
...,...,...,...,...
803,LakersReporter,"From the team: ""Dennis Schröder underwent succ...",2022-10-17 17:31:43,from the team dennis schröder underwent succes...
804,LakersReporter,Update on this: Troy Brown Jr. is more of a “d...,2022-10-17 13:56:03,update on this troy brown is more of situation...
805,LakersReporter,Darvin Ham said Dennis Schröder is expected to...,2022-10-17 13:36:39,darvin ham said dennis schröder is expected to...
806,LakersReporter,"When asked, AD mentioned Kendrick Nunn and Lon...",2022-10-17 13:24:00,when asked ad mentioned kendrick nunn and lonn...


### Every Pre-Game News

In [112]:
winr = []
allr = []
betr = []
gamedatar = []
predindex = []
for i in range(len(result4['Start time PT'])):

  if i == 0:
    for j in range(len(tweetr3['clean_tweet'])):
      if tweetr3['PT Date'][j] < result4['Start time PT'][i]:
        if tweetr3['clean_tweet'][j] != '':
  
          allr.append(tweetr3['clean_tweet'][j])
          winr.append(result4['Unnamed: 7'][i])
          betr.append(ats[i])
          gamedatar.append(result4.iloc[i,4:])
          predindex.append(j)
  if i > 0:

    for j in range(len(tweetr3['clean_tweet'])):
      
      if tweetr3['PT Date'][j]<result4['Start time PT'][i] and tweetr3['PT Date'][j]>result4['End time PT'][i-1]:
        if tweetr3['clean_tweet'][j] != '':

          allr.append(tweetr3['clean_tweet'][j])
          winr.append(result4['Unnamed: 7'][i])
          betr.append(ats[i])
          gamedatar.append(result4.iloc[i,4:])
          predindex.append(j)

In [84]:
len(allr)

756

In [107]:
datar = pd.DataFrame(index=range(len(allr)), columns=range(300))
nullindex = []
for index, text in enumerate(allr):
    vectors = []
    for i in range(len(word_tokenize(text))):
        try:
            vector = word_vectors[word_tokenize(text)[i]]
            vectors.append(vector)
        except KeyError:
            vectors.append(np.repeat(0,300))
    if vectors == []:
      print('Null',index)
      nullindex.append(index)
      vectors = np.repeat(0.0,300)
    else:
      vectors = np.nanmean(np.array(vectors),axis = 0).astype('float') 
    datar.iloc[index,:] = vectors
    
datar

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.041016,0.059919,0.053598,-0.01695,0.060721,0.016017,0.009626,-0.105294,0.028495,0.115984,...,-0.034215,-0.055136,-0.122811,0.058053,-0.010951,-0.002171,0.007246,0.017617,0.007289,0.005545
1,0.007357,0.00142,0.034627,0.049726,-0.030116,-0.001144,-0.00484,-0.099633,0.031305,0.071133,...,-0.041012,0.029144,-0.0884,0.032689,0.029092,-0.023539,-0.042816,-0.040214,0.01432,-0.042677
2,0.015524,-0.004865,-0.010376,0.089094,0.009821,0.001029,-0.027398,-0.071969,0.031555,0.000412,...,0.065674,0.017574,-0.07191,-0.036168,0.045238,-0.015214,-0.072366,-0.062631,-0.001116,0.002935
3,-0.02496,0.050022,0.072598,0.112237,0.004639,0.010539,0.031548,-0.097439,0.062486,0.058892,...,-0.074583,0.057282,-0.120443,0.140923,-0.036441,-0.037696,-0.012438,-0.056664,0.001817,0.034654
4,0.006678,0.006342,0.02195,0.053691,-0.017276,-0.027146,0.046341,-0.067137,0.06953,0.035422,...,0.019009,-0.002998,-0.077916,0.014271,0.021175,0.015287,-0.029796,-0.05691,-0.018387,-0.047999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,0.035966,0.0,-0.012016,0.014891,-0.0611,-0.000545,-0.006362,-0.119869,0.056474,0.068793,...,0.042614,0.077787,-0.084151,0.010271,0.05993,0.00465,-0.040739,0.03295,-0.011436,0.010571
752,0.035869,0.044027,0.038454,0.091914,-0.021119,-0.025054,0.048945,-0.081043,0.038388,0.046546,...,-0.04085,0.05376,-0.058899,-0.008421,-0.0237,0.037581,0.018714,-0.059574,-0.013398,-0.005173
753,-0.012149,0.007772,0.049118,0.069704,-0.033223,0.012276,0.046524,-0.056658,0.082464,0.031347,...,-0.022575,0.002338,-0.062248,0.023986,0.026627,-0.012484,-0.037589,-0.054405,0.021426,-0.008199
754,-0.031433,0.016098,0.061164,0.077423,-0.109756,-0.062103,-0.046371,-0.09079,0.072174,0.003189,...,-0.035553,-0.028091,0.019829,-0.103455,0.04422,0.031757,-0.097237,-0.047325,0.077545,-0.023123


In [86]:
label0r = []
for i in range(len(winr)):
  if winr[i] == 'W':
    label0r.append(int(1))
  if winr[i] == 'L':
    label0r.append(int(0))
label0r = np.array(label0r)
label0r.shape

(756,)

In [87]:
gameframer = pd.DataFrame(gamedatar)

In [88]:
gameframer.index = range(756)
datar.index = range(756)

In [89]:
datar2 = pd.concat([datar,gameframer],axis = 1)
datar2['ATS'] = betr
datar2

,0,1,2,3,4,5,6,7,8,9,...,Team.7,Opponent,Opponent.1,Opponent.2,Opponent.3,Opponent.4,Opponent.5,Opponent.6,Opponent.7,ATS
0,0.041016,0.059919,0.053598,-0.01695,0.060721,0.016017,0.009626,-0.105294,0.028495,0.115984,...,21,0.455,0.356,23,48,31,11,4,18,7.5
1,0.007357,0.00142,0.034627,0.049726,-0.030116,-0.001144,-0.00484,-0.099633,0.031305,0.071133,...,21,0.455,0.356,23,48,31,11,4,18,7.5
2,0.015524,-0.004865,-0.010376,0.089094,0.009821,0.001029,-0.027398,-0.071969,0.031555,0.000412,...,21,0.455,0.356,23,48,31,11,4,18,7.5
3,-0.02496,0.050022,0.072598,0.112237,0.004639,0.010539,0.031548,-0.097439,0.062486,0.058892,...,21,0.455,0.356,23,48,31,11,4,18,7.5
4,0.006678,0.006342,0.02195,0.053691,-0.017276,-0.027146,0.046341,-0.067137,0.06953,0.035422,...,21,0.455,0.356,23,48,31,11,4,18,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,0.035966,0.0,-0.012016,0.014891,-0.0611,-0.000545,-0.006362,-0.119869,0.056474,0.068793,...,15,0.413,0.25,14,47,23,7,10,9,-16.5
752,0.035869,0.044027,0.038454,0.091914,-0.021119,-0.025054,0.048945,-0.081043,0.038388,0.046546,...,15,0.413,0.25,14,47,23,7,10,9,-16.5
753,-0.012149,0.007772,0.049118,0.069704,-0.033223,0.012276,0.046524,-0.056658,0.082464,0.031347,...,15,0.413,0.25,14,47,23,7,10,9,-16.5
754,-0.031433,0.016098,0.061164,0.077423,-0.109756,-0.062103,-0.046371,-0.09079,0.072174,0.003189,...,15,0.413,0.25,14,47,23,7,10,9,-16.5


In [90]:
datar2 = np.array(datar2)
datar2.shape

(756, 317)

## Modeling CNN

In [91]:
#LakersNation
data3 = data2.astype('float')
#LakersReporter
datar2 = datar2.astype('float')

In [167]:
#NN imports
from keras.layers import Dense, Input, GlobalMaxPooling1D, Concatenate, Conv1D, MaxPooling1D
from keras.models import Model

input1 = Input((300, 1))
model_1D = Conv1D(128, 5, activation='relu')(input1)
model_1D = MaxPooling1D(5)(model_1D)
model_1D = Conv1D(128, 5, activation='relu')(model_1D)
model_1D = MaxPooling1D(5)(model_1D)
model_1D = Conv1D(128, 5, activation='relu')(model_1D)
model_1D = GlobalMaxPooling1D()(model_1D)
model_1D = Dense(128, activation='relu')(model_1D)
model_1D = Dense(50, activation='relu')(model_1D)
model_1D = Dense(15)(model_1D)

input2 = Input((17, ))
model_2 = Dense(10, activation='relu')(input2)
model_2 = Dense(8, activation='relu')(model_2)

merged = Concatenate()([model_1D, model_2])
merged = Dense(15, activation='relu')(merged)
merged = Dense(5, activation='relu')(merged)
output = Dense(1, activation='sigmoid')(merged)

model_final = Model(inputs=[input1, input2], outputs=[output])

model_final.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_final.summary()



Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 300, 1)]     0           []                               
                                                                                                  
 conv1d_6 (Conv1D)              (None, 296, 128)     768         ['input_5[0][0]']                
                                                                                                  
 max_pooling1d_4 (MaxPooling1D)  (None, 59, 128)     0           ['conv1d_6[0][0]']               
                                                                                                  
 conv1d_7 (Conv1D)              (None, 55, 128)      82048       ['max_pooling1d_4[0][0]']        
                                                                                            

In [168]:
#LakersNation
model_final.fit([data3[:,0:300],data3[:,300:]], label0, epochs=8, validation_split=0.2)

Epoch 1/8
111/111 [==============================] - 6s 26ms/step - loss: 0.6784 - accuracy: 0.5871 - val_loss: 0.7341 - val_accuracy: 0.2997
Epoch 2/8
111/111 [==============================] - 3s 23ms/step - loss: 0.6566 - accuracy: 0.6266 - val_loss: 0.6973 - val_accuracy: 0.5074
Epoch 3/8
111/111 [==============================] - 3s 23ms/step - loss: 0.6296 - accuracy: 0.6566 - val_loss: 0.6911 - val_accuracy: 0.5482
Epoch 4/8
111/111 [==============================] - 3s 26ms/step - loss: 0.5990 - accuracy: 0.7032 - val_loss: 0.6771 - val_accuracy: 0.5448
Epoch 5/8
111/111 [==============================] - 3s 25ms/step - loss: 0.5429 - accuracy: 0.7582 - val_loss: 0.6767 - val_accuracy: 0.5619
Epoch 6/8
111/111 [==============================] - 3s 24ms/step - loss: 0.4879 - accuracy: 0.7832 - val_loss: 0.6384 - val_accuracy: 0.5778
Epoch 7/8
111/111 [==============================] - 3s 25ms/step - loss: 0.4435 - accuracy: 0.8113 - val_loss: 0.6850 - val_accuracy: 0.5778
Epoch 

In [170]:
score, acc = model_final.evaluate([datar2[:,0:300],datar2[:,300:317]], label0r)

24/24 [==============================] - 0s 8ms/step - loss: 0.4595 - accuracy: 0.7725


In [171]:
predtext = tweetr3.loc[np.unique(predindex)]
predtext.index = range(756)

In [172]:
gameframer

,Team,Team.1,Team.2,Team.3,Team.4,Team.5,Team.6,Team.7,Opponent,Opponent.1,Opponent.2,Opponent.3,Opponent.4,Opponent.5,Opponent.6,Opponent.7
0,0.426,0.25,25,48,23,12,4,21,0.455,0.356,23,48,31,11,4,18
1,0.426,0.25,25,48,23,12,4,21,0.455,0.356,23,48,31,11,4,18
2,0.426,0.25,25,48,23,12,4,21,0.455,0.356,23,48,31,11,4,18
3,0.426,0.25,25,48,23,12,4,21,0.455,0.356,23,48,31,11,4,18
4,0.426,0.25,25,48,23,12,4,21,0.455,0.356,23,48,31,11,4,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,0.489,0.5,17,54,25,6,6,15,0.413,0.25,14,47,23,7,10,9
752,0.489,0.5,17,54,25,6,6,15,0.413,0.25,14,47,23,7,10,9
753,0.489,0.5,17,54,25,6,6,15,0.413,0.25,14,47,23,7,10,9
754,0.489,0.5,17,54,25,6,6,15,0.413,0.25,14,47,23,7,10,9


In [176]:
def textpredict(index):
    range2 = range(index,index+1)
    
    predvalue = model_final.predict([datar2[range2,0:300],datar2[range2,300:317]],verbose = 0)
    for index , i in enumerate(range2):
        if predvalue[index]> 0.5:
            print('Input text:',predtext.loc[i][3],'\nPredict result: Win')
        else:
            print('Input text:',predtext.loc[i][3],'\nPredict result: Loss')
    

In [177]:
textpredict(564)

Input text: austin reaves is up to 22 point season high on just 10 field goal attempt with and made ft lal lead with on the clock 
Predict result: Win


In [178]:
textpredict(123)

Input text: lal up after tough bucket from ad through draymond stop and schröder free throw with 29 second to play fantastic game from on both end 39 point board assist block 
Predict result: Win


In [179]:
textpredict(700)

Input text: anthony davis low back soreness is going to try to play darvin ham said he ll warm up and the plan is that he ll start 
Predict result: Loss
